# Adiabatic Quantum Computing and Quantum Annealing

When we talk about quantum computing, we actually talk about several different paradigms.
The most common one is gate-model quantum computing. In this case, gates are applied on qubit registers to perform arbitrary transformations of quantum states made up of qubits. 
    The second most common paradigm is quantum annealing. This paradigm is often also referred
to as adiabatic quantum computing, although there are subtle differences. Quantum annealing
solves a more specific problem -- universality is not a requirement -- which makes it an
easier, albeit still difficult engineering challenge to scale it up. The most prominent realisation is made by D-Wave Systems with more than 5000 qubits.


Gate-model quantum computing is conceptually easier to understand: it is the generalization
of digital computing. Instead of deterministic logical operations of bit strings, we have deterministic
transformations of (quantum) probability distributions over bit strings. We briefly
discuss the adiabatic theorem, which provides the foundation why quantum annealing would
work at all.

## 1 Adiabatic theorem adiabatic quantum computing

An adiabatic process means that conditions change slowly enough for the system to adapt to
the new configuration. For instance, in a quantum mechanical system, we can start from some
Hamiltonian $H_0$ and slowly change it to some other Hamiltonian $H_1$. The simplest change could
be a linear schedule:
$$ 
H(t) = (1 - t)H_0 + t\cdot H_1,
$$
for $t \in [0, 1]$ on some time scale. This Hamiltonian depends on time, so solving the Schrödinger
equation is considerably more complicated. The adiabatic theorem says that if the change in the
time-dependent Hamiltonian occurs slowly, the resulting dynamics remain simple: starting close
to an eigenstate, the system remains close to an eigenstate. This implies that if the system started
in the ground state, if certain conditions are met, the system stays in the ground state.
We call the energy difference between the ground state and the first excited state the gap. If
$H(t)$ has a nonnegative gap for each t during the transition and the change happens slowly, then
the system stays in the ground state. If we denote the time-dependent gap by $Δ(t)$, a course
approximation of the speed limit scales as $1/ min(Δ(t))^2$.
This theorem allows something highly unusual. We can reach the ground state of an easy-
to-solve quantum many body system, and change the Hamiltonian to a system we are interested
in. For instance, we could start with the Hamiltonian $\Sigma_i \sigma_i^X$
-- its ground state is just the equal superposition. Let’s see this on two sites:

In [17]:
!pip install dwave-ocean-sdk
import numpy as np
np.set_printoptions(precision=3,suppress=True)
             
X = np.array([[0, 1], [1, 0]])
IX = np.kron(np.eye(2), X)
XI = np.kron(X, np.eye(2))
H_0 = - (IX + XI)
E, v = np.linalg.eigh(H_0)
print("Eigenvalues:", E)
print("Eigenstate for lowest eigenvalue", v[:, 0])

Eigenvalues: [-2. -0.  0.  2.]
Eigenstate for lowest eigenvalue [-0.5 -0.5 -0.5 -0.5]


Note than the energy gap is non-negative and quite large.

Then we could turn this Hamiltonian slowly into a classical Ising model and read out the
global solution.
Adiabatic quantum computation exploits this phenomenon and it is able to perform universal
calculations with the final Hamiltonian being $H = -\Sigma_{<i,j>}J_{i,j}\sigma_i^Z\sigma_j^Z -\Sigma_i h_i\sigma_i^Z  -\Sigma_{<i,j>}g_{i,j}\sigma_i^X\sigma_j^X$

Note that is not the transverse-field Ising model: the last term is an X-X interaction. If a quantum
computer respects the speed limit, guarantees the finite gap, and implements this Hamiltonian,
then it is equivalent to the gate model with some overhead.

## 2 Quantum annealing
A theoretical obstacle to adiabatic quantum computing is that calculating the speed limit is clearly
not trivial; in fact, it is harder than solving the original problem of finding the ground state of some
Hamiltonian of interest. Engineering constraints also apply: the qubits decohere, the environment
has finite temperature, and so on. Quantum annealing drops the strict requirements and instead
of respecting speed limits, it repeats the transition (the annealing) over and over again. Having
collected a number of samples, we pick the spin configuration with the lowest energy as our
solution. There is no guarantee that this is the ground state.
Quantum annealing has a slightly different software stack than gate-model quantum computers.
Instead of a quantum circuit, the level of abstraction is the classical Ising model -- the problem
we are interested in solving must be in this form. Then, just like superconducting gate-model
quantum computers, superconducting quantum annealers also suffer from limited connectivity.
In this case, it means that if our problem’s connectivity does not match that of the hardware, we
have to find a graph minor embedding. This will combine several physical qubits into a logical
qubit.
A possible classical solver for the Ising model is the simulated annealer. D-Wave Ocean software suite offer ways to implement and solve an Ising problem. Let's see a simple example:

In [18]:
# simple Ising model and lowest energy 
import dimod
J = {(0, 1): 1.0, (1, 2): -1.0}
h = {0:0, 1:0, 2:0}
model = dimod.BinaryQuadraticModel(h, J, 0.0, dimod.SPIN)
sampler = dimod.SimulatedAnnealingSampler()
response = sampler.sample(model, num_reads=10)
print("Energy of samples:")
print([solution.energy for solution in response.data()])

Energy of samples:
[-2.0, -2.0, -2.0, -2.0, -2.0, -2.0, -2.0, -2.0, -2.0, -2.0]


In an adiabatic process, conditions change slowly enough for the system to
adapt to the new configuration. The speed of change heavily depends on the gap, that is, the difference between the ground state energy and the first excited state of all Hamiltonians $H(t)$, $t \in [0, 1]$.
It is easy to craft a Hamiltonian where this gap is small, so the speed limit has to be low. If
you take a classical Ising model with coupling strengths on vastly different scales, that is what
you get. For instance, calculate the gap (the difference between the smallest and second smallest
eigenvalue) of the Hamitonian $H_1 = -1000\sigma_1^Z\sigma_2^Z -0.1\sigma_2^Z\sigma_3^Z-0.5\sigma_1^Z$
acting on a three-qubit system
(the last linear term is there to make the ground state unique). Remember that since you have three qubits, the $\sigma_1^Z
\sigma_2^Z$ operator, for instance, actually means $\sigma_1^Z \otimes \sigma_2^Z \otimes \mathbb{1}$ 

In [19]:
# Calculate energy gap on a Ising model 
Z = np.array([[1, 0], [0, -1]])
#
ZZ = np.kron(Z, Z)
Z12I = np.kron(ZZ, np.eye(2))
ZI23 = np.kron(np.eye(2),ZZ)
Z1II = np.kron(Z, np.eye(4))
H = -1000*Z12I-0.1*ZI23-0.5*Z1II
print(H)

[[-1000.6    -0.     -0.     -0.     -0.     -0.     -0.     -0. ]
 [   -0.  -1000.4    -0.     -0.     -0.      0.     -0.     -0. ]
 [   -0.     -0.    999.6     0.     -0.     -0.      0.     -0. ]
 [   -0.     -0.      0.    999.4    -0.     -0.     -0.     -0. ]
 [   -0.     -0.     -0.     -0.   1000.4     0.      0.      0. ]
 [   -0.      0.     -0.     -0.      0.   1000.6     0.      0. ]
 [   -0.     -0.      0.     -0.      0.      0.   -999.4     0. ]
 [   -0.     -0.     -0.     -0.      0.      0.      0.   -999.6]]


In [20]:
E, v = np.linalg.eigh(H)
print(E)
print(v)
gap = E[1]-E[0]
print(gap)

[-1000.6 -1000.4  -999.6  -999.4   999.4   999.6  1000.4  1000.6]
[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]]
0.20000000000004547


this to the gap of the Hamiltonian $H_0 = \Sigma^3_{i=1} \sigma_i^X$. Again, calculate the value in a variable called gap.

In [21]:
X = np.array([[0, 1], [1, 0]])
#
X1II = np.kron(X, np.eye(4))
XI2I = np.kron(np.eye(2), np.kron(X, np.eye(2)))
XII3 = np.kron(np.eye(4), X)
H = -X1II -XI2I -XII3
print(H)
E, v = np.linalg.eigh(H)
print(E)
print(v)
gap = E[1]-E[0]
print(gap)
#

[[-0. -1. -1. -0. -1. -0. -0. -0.]
 [-1. -0. -0. -1. -0. -1. -0. -0.]
 [-1. -0. -0. -1. -0. -0. -1. -0.]
 [-0. -1. -1. -0. -0. -0. -0. -1.]
 [-1. -0. -0. -0. -0. -1. -1. -0.]
 [-0. -1. -0. -0. -1. -0. -0. -1.]
 [-0. -0. -1. -0. -1. -0. -0. -1.]
 [-0. -0. -0. -1. -0. -1. -1. -0.]]
[-3. -1. -1. -1.  1.  1.  1.  3.]
[[-0.354 -0.537 -0.204 -0.213  0.435 -0.431  0.     0.354]
 [-0.354 -0.261 -0.364  0.418 -0.432 -0.146  0.408 -0.354]
 [-0.354  0.092 -0.315 -0.517  0.247  0.54   0.149 -0.354]
 [-0.354  0.368 -0.476  0.114 -0.25   0.037 -0.558  0.354]
 [-0.354 -0.368  0.476 -0.114 -0.25   0.037 -0.558 -0.354]
 [-0.354 -0.092  0.315  0.517  0.247  0.54   0.149  0.354]
 [-0.354  0.261  0.364 -0.418 -0.432 -0.146  0.408  0.354]
 [-0.354  0.537  0.204  0.213  0.435 -0.431  0.    -0.354]]
1.9999999999999991


You can see that there is a vast difference in the gap between the two Hamiltonians. This could
be leveraged: for instance, the initial part of the annealing could go faster, since the gap is large,
and then slow down towards reaching the target Hamiltonian. The optimal annealing schedule is
a research topic on its own.

On a real quantum annealing device, we drop the stringent theoretical requirements
of following the adiabatic pathway and we repeat the transition over and over again.
Then we choose the lowest energy solution as our optimum.
The classical ’simulator’ for a quantum annealer is some heuristic solver of combinatorial optimization,
for instance, simulated annealing. We can use the dimod package to implement the Hamiltonian
with a small gap: $H_1 = -1000\sigma_1^Z\sigma_2^Z -0.1\sigma_2^Z\sigma_3^Z-0.5\sigma_1^Z$:

In [22]:
import dimod
#
J= {(0, 1): -1000, (1, 2): -0.1, (2, 3): 0}
h = {0:-0.5, 1:0, 2:0}
model = dimod.BinaryQuadraticModel(h, J, 0.0, dimod.SPIN)
sampler = dimod.SimulatedAnnealingSampler()
response = sampler.sample(model, num_reads=10)
print("Energy of samples:")
print([solution.energy for solution in response.data()])
#

Energy of samples:
[-1000.6, -1000.6, -1000.6, -1000.6, -999.6, -999.6, -999.6, -999.6, -999.6, -999.6]


Unlike in the case of a simple system, you often do not got the ground state!